# RedoxFlow Agent Demonstration

Import the agent class interface

In [5]:
from agent import RedoxFlow

Set REDOXGIT environmental variable to point to the root directory of the repo.

For demonstration, we keep all agent functionality within the repo (e.g., stored memory, resulting filepaths, etc.)

So, we require this environmental variable to be set.

In [6]:
import os
# Local Code Repo Location
os.environ["REDOXGIT"] = "/home/cameron/Documents/Github/RedoxFlow" # No / :)

In [7]:
# Initialize the agent
rf = RedoxFlow()

 ---

## Chemical & Simulation Script Generation

This prepare_scripts() method (1) generates reactants, (2) generates reduced product candidates for each reactant, (3) conformer searches both reactants and products, (4) saves the lowest energy reactant/product conformer pair in memory, and then (5) writes the simulation scripts with initial geometries.

In [8]:
agent_raw_output = rf.prepare_scripts(num_generated_reactants=10, # Number of generated reactants to save
                    max_heavy_atoms_per_reactant=8, # Max # of non-hydrogen atoms per reactant
                    min_uniq_elements=1, # Min of non-C elements allowed in the molecule
                    temperature=0.425, # Higher -> More unusual structures, more electronegatives
                    device='auto', # 'cpu', 'cuda:0' for GPUs, 'auto' to auto-determine
                    verbose=True) # Print out intermediate script preparation progress

________________________________________________________________________________________________
[REDUCTION CANDIDATE SEARCH]
 rc=0  cmd: /home/cameron/Documents/envs/chemflow/bin/python /home/cameron/Documents/Github/RedoxFlow/scripts/reduce.py --molecules /home/cameron/Documents/Github/RedoxFlow/memory/reactants_memory.csv --reactions /home/cameron/Documents/Github/RedoxFlow/scripts/reactions.json --out /home/cameron/Documents/Github/RedoxFlow/memory/reduction_candidates.csv --depth 2

--- Output ---
Wrote 15 rows to /home/cameron/Documents/Github/RedoxFlow/memory/reduction_candidates.csv




________________________________________________________________________________________________
[CONFORMER SEARCH]
 rc=0  cmd: /home/cameron/Documents/envs/chemflow/bin/python /home/cameron/Documents/Github/RedoxFlow/scripts/conformer.py --in /home/cameron/Documents/Github/RedoxFlow/memory/reduction_candidates.csv --out /home/cameron/Documents/Github/RedoxFlow/memory/products_memory.csv --max-c

 ---

## Variable Extraction from Simulations

The extract_variables() method takes successful simulations and auto-extracts the relevant thermodynamic variables for the redox potential calculation from the bulk simulation output.

We demonstrate variable extraction on a reactant/product pair of simulations prepared by the agent that we ran ahead of time.

In [4]:
test_sim_root = f'{os.environ.get("REDOXGIT")}/redox_calculation_test'

In [5]:
# Auto-extract thermodynamic variables from simulation output
rf.extract_variables(sim_root=test_sim_root, sim_ext='.out')

The variables are stored internally to each simulation folder (e.g., react_2_redox.csv). However, you can also call the variables live from the Jupyter notebook.

In [6]:
# We use the term 'miner' to refer to the text miners that extracts the variables
for miner in rf.miners['reactants']:
    print(miner.variableDF)

{'react_2.out':    System Energy (Hartree)  Temperature (K)  \
0              -326.489255           298.15   

   Zero-Point correction to Energy (kcal/mol)  \
0                                      97.299   

   Thermal correction to Energy (kcal/mol)  \
0                                  102.824   

   Thermal correction to Enthalpy (kcal/mol)  Total Entropy (cal/mol-K)  \
0                                    103.416                     91.319   

   Translational Entropy (cal/mol-K)   Rotational Entropy (cal/mol-K)  \
0                              39.732                          27.964   

   Vibrational Entropy (cal/mol-K)  Solvated System Energy (Hartree)  \
0                           23.622                        -326.50271   

   COSMO energy (Hartree)  Nuclear repulsion energy (Hartree)  \
0                0.049338                          312.629724   

   Exchange-Corr. energy (Hartree)  Coulomb energy (Hartree)  \
0                       -46.142545                468.53500

In [7]:
for miner in rf.miners['products']:
    print(miner.variableDF)

{'prod_2.out':    System Energy (Hartree)  Temperature (K)  \
0              -252.547921           298.15   

   Zero-Point correction to Energy (kcal/mol)  \
0                                     108.304   

   Thermal correction to Energy (kcal/mol)  \
0                                  113.691   

   Thermal correction to Enthalpy (kcal/mol)  Total Entropy (cal/mol-K)  \
0                                    114.284                     89.089   

   Translational Entropy (cal/mol-K)   Rotational Entropy (cal/mol-K)  \
0                              39.289                          27.233   

   Vibrational Entropy (cal/mol-K)  Solvated System Energy (Hartree)  \
0                           22.566                       -252.560288   

   COSMO energy (Hartree)  Nuclear repulsion energy (Hartree)  \
0                0.125202                          255.877751   

   Exchange-Corr. energy (Hartree)  Coulomb energy (Hartree)  \
0                       -38.385276                376.230239

If text-extraction failed (e.g., incomplete simulation or convergence criteria not met), these cases fail gracefully and are recorded. We provide this functionality to motivate next steps in automation for simulation crashing/relaunching.

Within our test simulation folder, 'react_3' and 'prod_3' deliberately contains incomplete simulation output to demonstrate this functionality.

In [8]:
# Print out which simulations failed thermodynamic variable extraction
print(rf.miner_errors)

{'reactants': [('/home/cameron/Documents/Github/RedoxFlow/redox_calculation_test/reactants/react_3/react_3.out', AttributeError("'NoneType' object has no attribute 'rstrip'"))], 'products': [('/home/cameron/Documents/Github/RedoxFlow/redox_calculation_test/products/prod_3/prod_3.out', AttributeError("'NoneType' object has no attribute 'rstrip'"))]}


 ---

## Calculating Redox Potential from Extracted Variables

In [9]:
rf.calculate_redox(n_reduction=2, # Number of participating electorns in PCET reaction (often matches number of hydrogens participating)
                   reference_state=0.0, # SHE reference state by default if zero
                   sim_root=test_sim_root,
                   sim_ext='.csv')

In [10]:
rf.redox_results

{'2': {'react_path': '/home/cameron/Documents/Github/RedoxFlow/redox_calculation_test/reactants/react_2/react_2_redox.csv',
  'prod_path': '/home/cameron/Documents/Github/RedoxFlow/redox_calculation_test/products/prod_2/prod_2_redox.csv',
  'h2_path': '/home/cameron/Documents/Github/RedoxFlow/redox_calculation_test/H2/H2_redox.csv',
  'n_reduction (# electrons)': 2,
  'reference_state': 0.0,
  'value (J/C or V)': -1.8479782175433048}}

In [11]:
# Failed redox potential calculations are
rf.redox_errors

[]